# Implementation
Author: Jørgen S. Dokken

In this tutorial, you will learn how to:
- Use a vector function space
- Create a constant boundary condition on a vector space
- Visualize cell wise constant functions
- Compute Von Mises stresses

## Test problem
As a test example, we will model a clamped beam deformed under its own weigth in 3D. This can be modeled, by setting the right-hand side body force per unit volume to $f=(0,0,-\rho g)$ with $\rho$ the density of the beam and $g$ the acceleration of gravity. The beam is box-shaped with length $L$ and has a square cross section of width $W$. we set $u=u_D=(0,0,0)$ at the clamped end, x=0. The rest of the boundary is traction free, that is, we set $T=0$. We start by defining the physical variables used in the program.

In [ ]:
# Scaled variable
L = 10#0.8
W = 2#0.2
#mu = 1
rho = 100
delta = W/L
gamma = 0.4*delta**2
#beta = 1.25
#lambda_ = beta
g = gamma

We then create the mesh, which will consist of hexahedral elements, along with the function space. We will use the convenience function `VectorFunctionSpace`. However, we also could have used `ufl`s functionality, creating a vector element `element = ufl.VectorElement("CG", mesh.ufl_cell(), 1)
`, and intitializing the function space as `V = dolfinx.fem.FunctionSpace(mesh, element)`.

In [ ]:
import numpy as np
import ufl

from mpi4py import MPI
from petsc4py.PETSc import ScalarType

from dolfinx import mesh, fem, plot, io
from dolfinx.fem import Function, FunctionSpace
import pyvista

num_elements_X = 20
num_elements_Y = 10

domain = mesh.create_rectangle(comm=MPI.COMM_WORLD,
                            points=((0.0, 0.0), (L, W)), n=(num_elements_X, num_elements_Y),
                            cell_type=mesh.CellType.quadrilateral)
V = fem.VectorFunctionSpace(domain, ("Lagrange", 1))


""" ElasticityConstant = fem.FunctionSpace(domain, ("CG", 1))

random_field = Randomfield(fct_space=ElasticityConstant, cov_name='squared_exp', mean=1, rho=0.5, sigma=1, k=150)

# solve eigenvalue problem
random_field.solve_covariance_EVP() # generalized eigenvalue problem of correlation matrix (M)

plt.figure()
plt.title("Eigenvalue decay")
plt.semilogy(range(len(random_field.lambdas)), sorted(random_field.lambdas, reverse=True))

# extract some modes of random field as fenics functions
random_field.k = 3
modes = random_field.modes(random_field.k, plot=False) """

In [1]:
from mpi4py import MPI
from dolfinx import mesh, fem, plot, io

domain = mesh.create_unit_square(MPI.COMM_WORLD, 40, 40, mesh.CellType.quadrilateral)
from dolfinx.fem import FunctionSpace
V = FunctionSpace(domain, ("CG", 1))

In [4]:
from randomfield_mod1 import Randomfield
#from relpgd.alib.logger import setup_annika_logger
import matplotlib.pyplot as plt
from mpi4py import MPI

""" def random_field_generator(domain, cov_name, mean, correlation_length, variance, no_eigen_values):
    field_function_space = fem.FunctionSpace(domain, ("CG", 1))
    random_field = Randomfield(field_function_space, cov_name, mean, correlation_length, variance, no_eigen_values)
    #random_field = Randomfield(fct_space=var_function_space, cov_name='squared_exp', mean=1, rho=0.5, sigma=1, k=10)
    #random_field.solve_covariance_EVP()
    return random_field """

#def random_field_generator(domain, cov_name, mean, correlation_length1, correlation_length2, variance, no_eigen_values):
def random_field_generator(domain, cov_name, mean, correlation_length1, variance, no_eigen_values):
    field_function_space = fem.FunctionSpace(domain, ("CG", 1))
    #random_field = Randomfield(field_function_space, cov_name, mean, correlation_length1, correlation_length2, variance, no_eigen_values)
    random_field = Randomfield(field_function_space, cov_name, mean, correlation_length1, variance, no_eigen_values)
    ####random_field = Randomfield(fct_space=var_function_space, cov_name='squared_exp', mean=1, rho=0.5, sigma=1, k=10)
    ####random_field.solve_covariance_EVP()
    return random_field

# domain, cov_name, mean, correlation_length, variance, no_eigen_values
# domain, cov_name, mean, correlation_length1, correlation_length2, variance, no_eigen_values

e_modulus = random_field_generator(domain,'squared_exp', 100, 0.05, 4, 3) 
e_modulus.create_random_field(_type='random')

""" poissons_ratio = random_field_generator(domain,'squared_exp', 0.2, 0.3, 0.05, 0, 3)
poissons_ratio.create_random_field(_type='random') """


[1.64687371e-17 1.84092880e-17 3.53403706e-17 ... 2.36835185e-01
 2.36835185e-01 2.45426177e-01] [[-7.34095785e-06  7.49443080e-05 -8.06757357e-06 ...  6.20934440e-02
  -1.72759146e-15 -2.19351401e-02]
 [ 1.08451983e-06 -1.45200152e-04 -7.89303402e-06 ...  9.62969916e-02
  -5.62978347e-04 -3.42167687e-02]
 [ 2.40142053e-05 -1.24298952e-04  3.51869543e-05 ...  9.62969916e-02
   5.62978347e-04 -3.42167687e-02]
 ...
 [ 4.28366610e-03  3.29007226e-03  1.27751707e-03 ... -9.62969916e-02
  -5.62978347e-04 -3.42167687e-02]
 [ 4.26159346e-03  3.38949978e-03  1.29498940e-03 ... -9.62969916e-02
   5.62978347e-04 -3.42167687e-02]
 [-2.67679346e-03 -2.08659165e-03 -8.16820210e-04 ... -6.20934440e-02
   4.24873347e-16 -2.19351401e-02]]


" poissons_ratio = random_field_generator(domain,'squared_exp', 0.2, 0.3, 0.05, 0, 3)\npoissons_ratio.create_random_field(_type='random') "

In [ ]:
pyvista.set_jupyter_backend("pythreejs")

e_modulus_topology, e_modulus_cell_types, e_modulus_geometry = plot.create_vtk_mesh(V)
print("Random field with %s" %(e_modulus.values))

e_modulus_grid = pyvista.UnstructuredGrid(e_modulus_topology, e_modulus_cell_types, e_modulus_geometry)
e_modulus_grid.point_data["Elasticity"] = e_modulus.field.vector[:]  
e_modulus_grid.set_active_scalars("Elasticity")
e_modulus_plotter = pyvista.Plotter()
e_modulus_plotter.add_mesh(e_modulus_grid, show_edges=True)
e_modulus_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    e_modulus_plotter.show()

In [ ]:
#pyvista.set_jupyter_backend("pythreejs")

poissonsratio_topology, poissonsratio_cell_types, poissonsratio_geometry = plot.create_vtk_mesh(V)
print("Random field with %s" %(poissons_ratio.values))

poissonsratio_grid = pyvista.UnstructuredGrid(poissonsratio_topology, poissonsratio_cell_types, poissonsratio_geometry)
poissonsratio_grid.point_data["Poissons"] = poissons_ratio.field.vector[:]  
poissonsratio_grid.set_active_scalars("Poissons")
poissonsratio_plotter = pyvista.Plotter()
poissonsratio_plotter.add_mesh(poissonsratio_grid, show_edges=True)
poissonsratio_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    poissonsratio_plotter.show()

## Boundary conditions
As we would like to clamp the boundary at $x=0$, we do this by using a marker function, which locate the facets where $x$ is close to zero by machine prescision.

In [ ]:

def clamped_boundary(x):                                    # fenics will individually call this function for every node and will note the true or false value.
    return np.isclose(x[0], 0)

fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(domain, fdim, clamped_boundary)

#print(boundary_facets)

u_D = np.array([0, 0], dtype=ScalarType)                    # array of zeros is supplied here as an input.

#fem.locate_dofs_geometrical(V, clamped_boundary) --- alternative for boundary facets
bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, boundary_facets), V)

#print(fem.locate_dofs_topological(V, fdim, boundary_facets)) # shows that dofs are actually nodes.
print(fem.locate_dofs_geometrical(V, clamped_boundary))
print(fem.locate_dofs_topological(V, fdim, boundary_facets))

As we want the traction $T$ over the remaining boundary to be $0$, we create a `dolfinx.Constant`

In [ ]:
T = fem.Constant(domain, ScalarType((0, 0)))    #no external load over the entire boundary. no problem in defining it over dirichlet boundary.

We also want to specify the integration measure $\mathrm{d}s$, which should be the integral over the boundary of our domain. We do this by using `ufl`, and its built in integration measures

In [ ]:
ds = ufl.Measure("ds", domain=domain)

## Variational formulation
We are now ready to create our variational formulation in close to mathematical syntax, as for the previous problems.

In [ ]:
#print(mu.field.vector[:])
#lambda_= fem.Constant(domain, lambda_.field.vector[:])
#mu = fem.Constant(domain, mu.field.vector[:])

def epsilon(u):
    return ufl.sym(ufl.grad(u)) # Equivalent to 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)

def lambda_(e_modulus, poissons_ratio):
    return (e_modulus.field*poissons_ratio.field)/((1+e_modulus.field)*(1-2*poissons_ratio.field))

def mu(e_modulus, poissons_ratio):
    return e_modulus.field/(2*(1+poissons_ratio.field))

def sigma(u):
    #return lambda_.field * ufl.nabla_div(u) * ufl.Identity(u.geometric_dimension()) + 2*mu.field*epsilon(u)
    #return (e_modulus.field*poissons_ratio.field)/((1+e_modulus.field)*(1-2*poissons_ratio.field))* ufl.nabla_div(u) * ufl.Identity(u.geometric_dimension()) + 2*e_modulus.field/(2*(1+poissons_ratio.field))*epsilon(u)
    return lambda_(e_modulus, poissons_ratio) * ufl.nabla_div(u) * ufl.Identity(u.geometric_dimension()) + 2*mu(e_modulus, poissons_ratio)*epsilon(u)

u = ufl.TrialFunction(V)  #u is the object of the argument class
v = ufl.TestFunction(V)
f = fem.Constant(domain, ScalarType((0, -rho*g)))
a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx
L = ufl.dot(f, v) * ufl.dx + ufl.dot(T, v) * ds

```{note}
Note that we used `nabla_grad` and optionally `nabla_div` for the variational formulation, as oposed to our previous usage of 
`div` and `grad`. This is because for scalar functions $\nabla u$ has a clear meaning
$\nabla u = \left(\frac{\partial u}{\partial x}, \frac{\partial u}{\partial y}, \frac{\partial u}{\partial z} \right)$.

However, if $u$ is vector valued, the meaning is less clear. Some sources define $\nabla u$ as a matrix with the elements $\frac{\partial u_j}{\partial x_i}$, while other  sources prefer 
$\frac{\partial u_i}{\partial x_j}$. In DOLFINx `grad(u)` is defined as the amtrix with element $\frac{\partial u_i}{\partial x_j}$. However, as it is common in continuum mechanics to use the other definition, `ufl` supplies us with `nabla_grad` for this purpose.
```
 
## Solve the linear variational problem
As in the previous demos, we assemble the matrix and right hand side vector and use PETSc to solve our variational problem

In [ ]:
problem = fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [ ]:
from dolfinx import geometry
bb_tree = geometry.BoundingBoxTree(domain, domain.topology.dim)
points=np.array([[0.1],[0.123], [0]])

cells = []
points_on_proc = []
# Find cells whose bounding-box collide with the the points
cell_candidates = geometry.compute_collisions(bb_tree, points.T)
print(type(cell_candidates))
# Choose one of the cells that contains the point
colliding_cells = geometry.compute_colliding_cells(domain, cell_candidates, points.T)
for i, point in enumerate(points.T):
    if len(colliding_cells.links(i))>0:
        points_on_proc.append(point)
        cells.append(colliding_cells.links(i)[0])

points_on_proc = np.array(points_on_proc, dtype=np.float64)
u_values = uh.eval(points_on_proc, cells)
print(u_values)

In [ ]:
#print(dir(cell_candidates.links(0)[0]))
print(cell_candidates)
print(colliding_cells)
#print(cell_candidates.array)
#print(cell_candidates.links(0)[0])
#print(points_on_proc)

In [ ]:
#print(uh.x.array.shape)
#print(uh.__dict__)
print(uh([1,0]))
print(dir(domain))
#Sprint(domain.topology.create_entities(3))
print(dir(uh))
#print((uh.x.array))

print(uh.eval([0,1,0],100))

## Visualization

As in the previous demos, we can either use Pyvista or Paraview for visualization. We start by using Pyvista. Instead of adding scalar values to the grid, we add vectors.

In [ ]:
from dolfinx import plot
import pyvista
pyvista.set_jupyter_backend("pythreejs")

topology, cell_types, geometry = plot.create_vtk_mesh(domain, domain.topology.dim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

""" plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    pyvista.start_xvfb()
    figure = plotter.screenshot("fundamentals_mesh.png") """

u_topology, u_cell_types, u_geometry = plot.create_vtk_mesh(V)
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.reshape((u_geometry.shape[0], 2))
print(u_geometry.shape[0])
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()

In [ ]:
#alt

import pyvista
pyvista.set_jupyter_backend("pythreejs")

# Create plotter and pyvista grid
p = pyvista.Plotter()
topology, cell_types, geometry = plot.create_vtk_mesh(V)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

# Attach vector values to grid and warp grid by vector
grid["u"] = uh.x.array.reshape((geometry.shape[0], 3))
actor_0 = p.add_mesh(grid, style="wireframe", color="k")
warped = grid.warp_by_vector("u", factor=1.5)
actor_1 = p.add_mesh(warped, show_edges=True)
p.show_axes()
if not pyvista.OFF_SCREEN:
   p.show()
else:
   pyvista.start_xvfb()
   figure_as_array = p.screenshot("deflection.png")

We could also use Paraview for visualizing this.
As explained in previous sections, we save the solution with `XDMFFile`.
After opening the file `deformation.xdmf` in Paraview and pressing `Apply`, one can press the `Warp by vector button` ![Warp by vector](warp_by_vector.png) or go through the top menu (`Filters->Alphabetical->Warp by Vector`) and press `Apply`. We can also change the color of the deformed beam by changing the value in the color menu ![color](color.png) from `Solid Color` to `Deformation`.

In [ ]:
print(help(domain.geometry))

In [ ]:
with io.XDMFFile(domain.comm, "2DCantilever_poissons_ratio.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    poissons_ratio.field.name = "poissons ratio"
    xdmf.write_function(poissons_ratio.field)

In [ ]:
print(type(e_modulus))
with io.XDMFFile(domain.comm, "2DCantilever_e_modulus.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    e_modulus.field.name = "e_Modulus"
    xdmf.write_function(e_modulus.field)

In [ ]:
with io.XDMFFile(domain.comm, "2DCantilever_random.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    uh.name = "Deformation"
    xdmf.write_function(uh)

## Stress computation
As soon as the displacement is computed, we can compute various stress measures. We will compute the von Mises stress defined as $\sigma_m=\sqrt{\frac{3}{2}s:s}$ where $s$ is the deviatoric stress tensor $s(u)=\sigma(u)-\frac{1}{3}\mathrm{tr}(\sigma(u))I$.

In [ ]:
s = sigma(uh) -1./3*ufl.tr(sigma(uh))*ufl.Identity(uh.geometric_dimension())
von_Mises = ufl.sqrt(3./2*ufl.inner(s, s))

The `von_Mises` variable is now an expression that must be projected into an appropriate function space so that we can visualize it. As `uh` is a linear combination of first order piecewise continuous functions, the von Mises stresses will be a cell-wise constant function.

In [ ]:
V_von_mises = fem.FunctionSpace(domain, ("DG", 0))
stress_expr = fem.Expression(von_Mises, V_von_mises.element.interpolation_points)
stresses = fem.Function(V_von_mises)
stresses.interpolate(stress_expr)

In the previous sections, we have only visualized first order Lagrangian functions. However, the Von Mises stresses are piecewise constant on each cell. Therefore, we modify our plotting routine slightly. The first thing we notice is that we  now set values for each cell, which has a one to one correspondence with the degrees of freedom in the function space.

In [ ]:
#topology, cell_types, geometry = plot.create_vtk_mesh(domain, domain.topology.dim)
#grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

#pyvista.set_jupyter_backend("pythreejs")

#plotter = pyvista.Plotter()
#plotter.add_mesh(grid, show_edges=True)
#plotter.view_xy()
#if not pyvista.OFF_SCREEN:
#    plotter.show()
#else:
#    pyvista.start_xvfb()
#    figure = plotter.screenshot("fundamentals_mesh.png")